In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

from time import time
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import random
%matplotlib inline 

In [8]:
def load_data():
    print('loading data...')
    start = time()
    X = np.loadtxt('data/cropped16.csv', delimiter=',', dtype=np.float32)
    Y = np.loadtxt('data/cropped50.csv', delimiter=',', dtype=np.float32)
    X.shape = (-1,16,16,3)
    Y.shape = (-1,7500)
    print('Job completed in: {} seconds'.format(time()-start))
    return X, Y

X,Y = load_data()

print(X.shape, Y.shape)

x = X/255
y = Y/255
n = 4100
x_train = x[:n]
x_test = x[n:]
y_train = y[:n]
y_test = y[n:]

#channel last

loading data...
Job completed in: 90.4001874923706 seconds
(5112, 16, 16, 3) (5112, 7500)


In [30]:
def model(x):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet',reuse=True):

        x = tf.reshape(x, shape=[-1, 16, 16, 3])
        conv1 = tf.layers.conv2d(x, 32, 3, activation=tf.nn.relu)
        
        fc1 = tf.contrib.layers.flatten(conv1)
        fc1 = tf.layers.dense(fc1, 1024, activation=tf.nn.relu)
        fc1 = tf.reshape(fc1, shape=[-1,32,32,1])
        #fc2 = tf.layers.dense(fc1, 3072, activation=tf.nn.relu)
        #fc2 = tf.reshape(fc1, shape=[-1,32,32,1])
    #with tf.variable_scope('ConvNet',reuse=True):
        conv2 = tf.layers.conv2d(fc1, 64, 3, activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(conv2, 128, 5, activation=tf.nn.relu)
        
        fc3 = tf.contrib.layers.flatten(conv3)
        fc3 = tf.layers.dense(fc3, 2500, activation=tf.nn.relu)
        out = tf.layers.dense(fc4, 7500, activation=tf.nn.relu)

    return out

In [10]:
def binary_cross_entropy(output, target, epsilon=1e-8, name='bce_loss'):
    """Computes binary cross entropy given `output`.
    For brevity, let `x = output`, `z = target`(prediction).  The binary cross entropy loss is
        loss(x, z) = - sum_i (x[i] * log(z[i]) + (1 - x[i]) * log(1 - z[i]))
    Parameters
    ----------
    output : tensor of type `float32` or `float64`.
    target : tensor of the same type and shape as `output`.
    epsilon : float
        A small value to avoid output is zero.
    name : string
        An optional name to attach to this layer.
    References
    -----------
    - `DRAW <https://github.com/ericjang/draw/blob/master/draw.py#L73>`_
    """
#     from tensorflow.python.framework import ops
#     with ops.op_scope([output, target], name, "bce_loss") as name:
#         output = ops.convert_to_tensor(output, name="preds")
#         target = ops.convert_to_tensor(targets, name="target")
    with tf.name_scope(name):
        return tf.reduce_mean(tf.reduce_sum(-(target * tf.log(output + epsilon) +
                              (1. - target) * tf.log(1. - output + epsilon)), axis=1))

def progress_bar(progress,n):
    for j in range(32):
        if j == 0: 
            continue
        if j == n:
            progress[j] = '>'
            break
        progress[j] = '='
    return progress

In [31]:
def train_neural_network(x_trian, x_test, y_train, y_test):
    #define x and y (include test set?)
    x = tf.placeholder(tf.float32, shape=[None, 16,16,3])
    y = tf.placeholder(tf.float32, shape=[None, 7500])
    epochs = 5
    batch_size = 32
    niter = len(x_train)/batch_size
    titer = len(x_test)/batch_size
    
    prediction = model(x)
    
    loss = binary_cross_entropy(y,prediction)
    optimizer = tf.train.AdagradOptimizer().minimize(loss)
    
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)
    
    for epoch in range(epochs):
        s = time()
        print('Epoch ',epoch+1,'/ epochs')
        progress = '[..............................]' #30
        tr_loss = 0
        val_loss = 0
        niter = len(x_train)/batch_size
        titer = len(x_test)/batch_size
        end = '\r'
        
        for i in range(niter):
            if i == (niter-1): end = ' '
            print(progress, end=end, flush=True)
            batch = tf.train.batch([x_train, y_train], batch_size)
            _,c = sess.run([optimizer,loss], feed_dict={x: batch[0], y: batch[1]})
            tr_loss += c/niter
            n = int(((i+1)/niter)*30)
            progress = progress_bar(progress, n)
                   
        for i in range(titer):
            testbatch = tf.train.batch([x_test, y_test], batch_size)
            cval = session.run([loss], feed_dict={x:testbatch[0], y:testbatch[1]})
            val_loss = cval/titer
        
        print('- {}s - loss: {:9f} - val_loss: {:9f}'.format(time()-s, tr_loss, val_loss))
    print("Optimization Finished")
    
    '''
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    '''
    

train_neural_network(x_train, x_test, y_train, y_test)

ValueError: Variable ConvNet/conv2d_1/kernel does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

In [ ]:
def model_fn(features, labels, mode):

    logits_train = conv_net(features, reuse=False, is_training=True)
    logits_test = conv_net(features, reuse=True,is_training=False)

    pred_ train = tf.argmax(logits_test, axis=1)
    pred_test = tf.nn.softmax(logits_test)

    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

        # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())

    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    # TF Estimators requires to return a EstimatorSpec, that specify the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [ ]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])

In [ ]:
def predict():
    decoded_imgs = autoencoder.predict(x_test)

    n = 8 

    plt.figure(figsize=(64, 64))
    k = 0
    for j in range(8):
        for i in range(n):
            ax = plt.subplot(8,8, j + 1 + 8*i)
            plt.imshow(X[k].T.reshape(32,32,3))
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            k += 1


    plt.show()